In [1]:
import pymysql
import pandas as pd

host_name = 'localhost'
host_port = 3306
username='root'
password=''
database_name='news_team_4'

db = pymysql.connect(
    host=host_name,
    port=host_port,
    user=username,
    password=password,
    db=database_name,
    charset='utf8'
)

SQL='SELECT * FROM tb_news_team_4'
df=pd.read_sql(SQL, db)

/var/folders/xr/x8smbq7d6l1g33fxnfy1j4b80000gn/T/ipykernel_8149/3598819719.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df=pd.read_sql(SQL, db)


In [2]:
df.shape

(1201941, 13)

In [ ]:
df.head(3)

In [3]:
print(df[df['ID']==169174][['MainCategory','SubCategory']])
print(df[df['ID']==1013437][['MainCategory','SubCategory']])
print(df[df['ID']==1044078][['MainCategory','SubCategory']])
print(df[df['ID']==391439][['MainCategory','SubCategory']])
print(df[df['ID']==1027247][['MainCategory','SubCategory']])

       MainCategory SubCategory
169173           사회       식품/의료
       MainCategory SubCategory
879769           세계      아시아/호주
       MainCategory SubCategory
910410           세계       세계 일반
       MainCategory SubCategory
257771           정치       국회/정당
       MainCategory SubCategory
893579           세계       세계 일반


In [5]:
print(df[df['ID']==639135]['Title'])
print(df[df['ID']==674310]['Title'])
print(df[df['ID']==667250]['Title'])
print(df[df['ID']==623156]['Title'])
print(df[df['ID']==655031]['Title'])
print(df[df['ID']==618203]['Title'])
print(df[df['ID']==588794]['Title'])
print('---------')
print(df[df['ID']==629312]['Title'])
print(df[df['ID']==629399]['Title'])
print(df[df['ID']==629823]['Title'])

505467    尹 대통령 지지율, 16%까지 곤두박질 ‘주요국 꼴찌’…非지지 최고 81% [나우,어스]
Name: Title, dtype: object
540642    팔순 맞는 바이든…중간선거 선방에 '대선은 여든부터' 통할까
Name: Title, dtype: object
533582    공화당에 하원 내주고도 바이든 "민주주의 지켜냈다"
Name: Title, dtype: object
489488    中 상하이 봉쇄 때보다 심각...시위·폭동까지
Name: Title, dtype: object
521363    '헤르손 결전' 임박...서방, '전차와 호크' 첫 지원
Name: Title, dtype: object
484535    대만 지방선거서 여당 참패…차이잉원 총통 당 주석직 사퇴
Name: Title, dtype: object
455126    “이걸로 삼성 크게 위협?” 중국 여성 손에 접는폰 ‘정체’ 알고보니
Name: Title, dtype: object
---------
495644    이란 대표팀, 월드컵서 국가 제창 거부로 '히잡 시위' 연대
Name: Title, dtype: object
495731    "한미일 외교차관, 北ICBM 발사 강력규탄·3자협력 중요성 확인"
Name: Title, dtype: object
496155    방역 고삐 죄는 중국‥곳곳서 도심 봉쇄
Name: Title, dtype: object


In [6]:
import pandas as pd # raw dataset
from surprise import SVD, accuracy # SVD model, 평가
from surprise import Reader, Dataset # SVD model의 dataset

/Users/choibowon/opt/anaconda3/envs/project1/lib/python3.10/site-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 0.996-ko-0.9.2 is an invalid version and will not be supported in a future release
  warnings.warn(


In [ ]:
import pandas as pd
import re
from konlpy.tag import Mecab
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
from tqdm import tqdm  # 반복문 진행상황 보기
from sklearn.metrics.pairwise import cosine_similarity
from numpy import dot
from numpy.linalg import norm
import numpy as np
from gensim.models.callbacks import CallbackAny2Vec

class MonitorCallback(CallbackAny2Vec):
    def __init__(self):
        self.epoch = 0
        self.last = time.time()

    def on_epoch_end(self, model):
        print('epoch ends {}: {}'.format(self.epoch, time.time() - self.last))
        self.epoch += 1
        self.last = time.time()


# to see all data
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 450)
pd.set_option('display.colheader_justify', 'left')

columns = ['DomainID', 'MainCategory', 'SubCategory', 'WritedAt', 'Title', 'Content', 'URL', 'PhotoURL', 'Writer',
           'Press', 'Stickers', 'UserID', 'Comment_content', 'Comment_WritedAt', 'UserName']

# df_news_foreign_dataset = pd.read_csv('news_4.csv')

def preprocessing_korean(data):
    fi_da = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ.*\\s]", '', data).replace('\xa0', '')
    fi_da = fi_da.replace("..", '').split('.')
    return fi_da

def make_doc2vec_data(df: pd.DataFrame):
    df.dropna(axis='index', inplace=True)
    tagged_corpus_list = []
    for idx, row in tqdm(df.iterrows(), total=len(df)):
        mecab = Mecab()
        text = df['Content']
        tag = row['Title']
        tagged_corpus_list.append(TaggedDocument(tags=[tag], words=mecab.morphs(text)))
        if idx > 1000:
            break
    return tagged_corpus_list

tagged_corpus_list = make_doc2vec_data(df)
model = Doc2Vec(vector_size=100, alpha=0.025, min_alpha=0.01, workers=6, window=2, callbacks=[MonitorCallback()])
model.build_vocab(tagged_corpus_list)
model.train(tagged_corpus_list, total_examples=model.corpus_count, epochs=50)

def cos_simimlarity(first_docu, second_docu):
    return dot(first_docu, second_docu) / (
            norm(first_docu) * norm(second_docu))

df.dropna(subset=['Content'], inplace=True)
mdoel_size = len(loaded_model.dv)

similarity_scores = []
similarity_scores_list = []
for idx, content_first in enumerate(tqdm(loaded_model.dv)):
    for idx_second in range(0, mdoel_size - 1):
        try:
            # similarity_scores.append(cos_simimlarity(content_first, loaded_model.dv[idx_second]))
            cosine_result = cosine_similarity([loaded_model.dv[idx].T], [loaded_model.dv[idx_second]])
            similarity_scores.append(*cosine_result)
            # print("cosine_result : " , idx, idx_second, *cosine_result)
        except Exception as e:
            print("Exception : ", idx, ":", idx_second, ":", e)
    similarity_scores_list.append(similarity_scores)

In [15]:
from konlpy.tag import Mecab
import re

def preprocessing(data):
    text=re.sub("[①②③④⑤⑥⑦⑧⑨⑩○●〈〉]",'',data).replace('\xa0','')
    return text

def mecab_morphs(data):
    pos_list = {'NNG', 'VA', 'VAX', 'MAG', 'NNP', 'VV+EC','VV+ETM'}
    mecab = Mecab()
    data_token = mecab.morphs(data)
    f = open('./stopwords-ko.txt', 'r')
    lines = f.readlines()
    
    stopwords_list= []
    for l in lines:
        l = l.replace('\n','')
        stopwords_list.append(l)
    
    result = []
    for token in data_token:
        if token not in stopwords_list:
            result.append(token)
    
    token_result = ''.join(result)
    pos_token = mecab.pos(token_result)
    pos_result = []
    for p in pos_token:
        if p[1] in pos_list:
            pos_result.append(p[0])
    pos_result = list(set(pos_result))
    
    return pos_result

def article_preprocessing(news_df): 
    '''
    데이터 프레임 각 기사 내용 전처리
    '''
    c= news_df['Content']
    content = pd.DataFrame(data=c, columns=['Content'])
    content['Content'].apply(lambda x: str(x))
    content['Content'] = content['Content'].apply(lambda x : preprocessing(x))
    content['Content'] = content['Content'].apply(lambda x : mecab_morphs(x))
    news_df['Content'] = content['Content'].apply(lambda x : ' '.join(x))

    return news_df

# news_df = pd.read_csv('./Article_news.csv') #뉴스 기사 있는 csv파일
result_df = article_preprocessing(df[600972:]) #원래 데이터 프레임 + 전처리된 내용
result_df.to_csv('./news_prep2.csv',index=False)

/var/folders/xr/x8smbq7d6l1g33fxnfy1j4b80000gn/T/ipykernel_16962/3335904482.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_df['Content'] = content['Content'].apply(lambda x : ' '.join(x))


In [14]:
result_df['Content'].iloc[100:]

451225     먹통 재발 추구 방지 대책 진행 원인 남궁훈 데이터 바라 발전 중요 이렇게 다시 언...
455695     기능 평가 뒤 한계 충실 브랜드 발광 칼 쉽 흑백 난이도 호평 주목 샤프 대용량 과...
432293     자금 큰손 우량 쉽 유예 높 회사채 대비 줄 보험 기금 사채 유동 치우 수요 당국 ...
692660     기능 앞서 오후 공공 망 개시 정차 대책 재난 경계 추진 단속 지역 교통부 반 임무...
270408     앞서 평가 뒤 공백 인물 예측 깊 관악 갖 나타낼 정심 장엄 준비 전국 기본 총선 ...
                                 ...                        
275075     드린 고차방정식 언론 느끼 저항감 존중 드리 설치 종교 기후 구결 낙태 환경 혼문 ...
195078     연안 울진 확보 독 생물 조종 높 다양 확대 보전 전국 중요 설치 기후 진봉 지역 ...
1044987    느 평가 국제 음일 심기 방지 중립 법인 노인 체계 고강 공동체 보전 청렴 기구 전...
941084     종합 건달 국회 차차 오후 소속 행정 지난달 방청 초동 없 관계 상황실 자료 그대로...
659665     건립 축하 하반기 원인 세대 임신 채택 구조 전국 기금 발전 기본 최대한 정도 지역...
Name: Content, Length: 5910, dtype: object

In [8]:
import numpy as np

df_1=pd.read_csv('news_prep1.csv')
df_2=pd.read_csv('news_prep2.csv')

result_df = pd.concat([df_1, df_2])
result_df['CosSimilarity']=0

result_df.shape

# result_df.to_csv("news_prep.csv", index=False)

/var/folders/xr/x8smbq7d6l1g33fxnfy1j4b80000gn/T/ipykernel_5500/1927856447.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_1=pd.read_csv('news_prep1.csv')
/var/folders/xr/x8smbq7d6l1g33fxnfy1j4b80000gn/T/ipykernel_5500/1927856447.py:4: DtypeWarning: Columns (8,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2=pd.read_csv('news_prep2.csv')


(1201941, 13)

In [9]:
result_df.to_csv("news_prep.csv", index=False)

In [10]:
result_df.shape

(1201941, 13)

In [14]:
SQL='SELECT * FROM tb_news_team_4'
df=pd.read_sql(SQL, db)
df.shape

/var/folders/xr/x8smbq7d6l1g33fxnfy1j4b80000gn/T/ipykernel_5500/2799023512.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df=pd.read_sql(SQL, db)


(1201941, 13)

In [7]:
import pandas as pd
import re
from konlpy.tag import Mecab
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
from tqdm import tqdm  # 반복문 진행상황 보기

# to see all data
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 450)
pd.set_option('display.colheader_justify', 'left')

columns = ['DomainID', 'MainCategory', 'SubCategory', 'WritedAt', 'Title', 'Content', 'URL', 'PhotoURL', 'Writer',
           'Press', 'Stickers', 'UserID', 'Comment_content', 'Comment_WritedAt', 'UserName']

df_news_foreign_dataset = df

mecab = Mecab()

df_news_foreign_dataset.dropna(axis='index', inplace=True)

tagged_corpus_list = []
for idx, row in tqdm(df_news_foreign_dataset.iterrows(), total=len(df_news_foreign_dataset)):
    text = row['Content']
    tag = row['Title']
    tagged_corpus_list.append(TaggedDocument(tags=[tag], words=mecab.morphs(text)))
    if idx > 1000:
        break
    
model = Doc2Vec(vector_size=100, alpha=0.025, min_alpha=0.01, workers=6, window=2)
model.build_vocab(tagged_corpus_list)
model.train(tagged_corpus_list, total_examples=model.corpus_count, epochs=50)
# model.infer_vector(df_news_foreign_dataset['Content'])
print(model.dv.most_similar("싸이에 버림받은 코인 '싸이클럽' 결국 국내 상폐", topn=20))
print()
print(model.dv.most_similar("싸이에 버림받은 코인 '싸이클럽' 결국 국내 상폐"))
print()
print(model.dv.most_similar("니콘이미징코리아, 고진영 프로와 6년 연속 쿨샷 전속모델 계약"))

In [ ]:
# result_df.to_csv("news_prep.csv", index=False)

In [3]:
# comm_df=pd.read_csv('News_4_Comment.csv')

# comm_df['CosSimilarity']= np.nan

# comm_df.to_csv("comm_prep.csv", index=False)